In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import os
from typing import List

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from sklearn.cluster import KMeans
from multiprocessing import Pool as MultiprocessingPool, cpu_count

from src.kinematics import calculate_speed_and_direction

pd.set_option("display.max_columns", None)

/Users/kniu91/Documents/projects/bdb-26/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ============================================================================
# CONFIG
# ============================================================================

class Config:
    DATA_DIR = Path("./data")
    OUTPUT_DIR = Path("./outputs")
    OUTPUT_DIR.mkdir(exist_ok=True)
    
    SEED = 42
    N_FOLDS = 5
    BATCH_SIZE = 256
    EPOCHS = 60
    PATIENCE = 30
    LEARNING_RATE = 1e-4
    
    WINDOW_SIZE = 10
    HIDDEN_DIM = 128
    MAX_FUTURE_HORIZON = 94
    
    FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
    FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3
    
    K_NEIGH = 6
    RADIUS = 30.0
    TAU = 8.0
    N_ROUTE_CLUSTERS = 7
    
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(Config.SEED)



In [4]:
c = pd.read_csv('./outputs/defender_impact_log_wr.csv')
# c.columns = ['game_id', 'play_id', 'nfl_id', 'baseline_prob', 'real_prob', 'delta', 'fold']
# c.head()
c = c.merge(supplementary_data[['game_id','play_id','pass_result', 'yards_gained', 'season','week','home_team_abbr','visitor_team_abbr','play_description', 'quarter','game_clock']],
        on=['game_id','play_id'], how='left')
# c[(c['game_id'] == 2023091711) & (c['play_id'] == 1082)]
c.sort_values('delta', ascending = False).head(20)

NameError: name 'supplementary_data' is not defined